In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import string
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Flatten, GlobalMaxPooling1D
from tensorflow.keras.models import Model
import random

In [2]:
intents = json.loads(open('intents1.json').read())

In [3]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you?',
    'Is anyone there?',
    'Hello',
    'Good day',
    "What's up",
    'how are ya',
    'heyy',
    'whatsup',
    '??? ??? ??'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'see you',
    'bye bye',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Bye',
    'Have a Good day',
    'talk to you later',
    'ttyl',
    'i got to go',
    'gtg'],
   'responses': ['Sad to see you go :(',
    'Talk to you later',
    'Goodbye!',
    'Come back soon'],
   'context_set': ''},
  {'tag': 'creator',
   'patterns': ['what is the name of your developers',
    'what is the name of your creators',
    'what is the name of the developers',
    'what is the name of the creators',
    'who created you',
    'your developers',
    'your creators',
    'who are your developers',
    'developers',
  

In [4]:
tags = []
inputs = []
responses = {}
for intent in intents['intents']:
    responses[intent['tag']] = intent["responses"]
    for lines in intent['patterns']:
        inputs.append(lines)
        tags.append(intent['tag'])


print("this is tags",tags)
#tags:liste des classes
print("this is input",inputs)
#inputs:liste des input
print("this is responses",responses)
#{'greeting': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'],
#'goodbye': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!', 'Come back soon']}

this is tags ['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'creator', 'creator', 'creator', 'creator', 'creator', 'creator', 'creator', 'creator', 'creator', 'creator', 'creator', 'creator', 'creator', 'creator', 'creator', 'creator', 'name', 'name', 'name', 'name', 'name', 'name', 'name', 'name', 'name', 'name', 'name', 'name', 'name', 'hours', 'hours', 'hours', 'hours', 'hours', 'hours', 'hours', 'hours', 'hours', 'hours', 'hours', 'hours', 'hours', 'hours', 'hours', 'hours', 'hours', 'number', 'number', 'number', 'number', 'number', 'number', 'number', 'number', 'number', 'number', 'number', 'number', 'number', 'number', 'number', 'course', 'course', 'course', 'course', 'course', 'course', 'course', 'course', 'course', 'course', 'course', 'course', 'course', 'course', 'course', '

In [5]:
print(len(inputs))
print(len(tags))

405
405


In [8]:
data = pd.DataFrame({"patterns":inputs,
                    "tags":tags})

In [9]:
data


,patterns,tags
0,Hi,greeting
1,How are you?,greeting
2,Is anyone there?,greeting
3,Hello,greeting
4,Good day,greeting
...,...,...
400,ragging history,ragging
401,ragging incidents,ragging
402,hod,hod
403,hod name,hod


# extraction des ponctuations et mettre tous les mots en miniscule 

In [10]:
data["patterns"] = data["patterns"].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])

In [11]:
data["patterns"] = data["patterns"].apply(lambda wrd: ''.join(wrd))

In [12]:
data

,patterns,tags
0,hi,greeting
1,how are you,greeting
2,is anyone there,greeting
3,hello,greeting
4,good day,greeting
...,...,...
400,ragging history,ragging
401,ragging incidents,ragging
402,hod,hod
403,hod name,hod


# Tokenizing and Padding

In [13]:
# num_words = 2000: le tokenizer ne conservera que les 2000 mots les plus fréquents dans vos données,
#en ignorant les mots moins fréquents.
tokenizer = Tokenizer(num_words = 2000)
# le tokenizer apprendra le vocabulaire à partir de ces données et attribuera un indice unique 
#à chaque mot dans le vocabulaire
tokenizer.fit_on_texts(data["patterns"])
#Chaque mot du texte est remplacé par son indice correspondant dans le vocabulaire 
#que le tokenizer a appris
train = tokenizer.texts_to_sequences(data["patterns"])

train

[[160],
 [12, 9, 5],
 [1, 161, 24],
 [162],
 [67, 110],
 [68, 111],
 [12, 9, 163],
 [164],
 [165],
 [],
 [166],
 [112, 5],
 [88, 88],
 [112, 5, 113],
 [167],
 [15, 89, 168],
 [88],
 [29, 90, 67, 110],
 [169, 18, 5, 113],
 [170],
 [15, 171, 18, 172],
 [173],
 [3, 1, 4, 13, 6, 16, 59],
 [3, 1, 4, 13, 6, 16, 69],
 [3, 1, 4, 13, 6, 4, 59],
 [3, 1, 4, 13, 6, 4, 69],
 [19, 114, 5],
 [16, 59],
 [16, 69],
 [19, 9, 16, 59],
 [59],
 [5, 9, 91, 115],
 [5, 9, 91, 115, 174],
 [19, 114, 5],
 [19, 175, 5],
 [69],
 [19, 91, 5],
 [19, 176, 5],
 [13],
 [16, 13],
 [25, 5, 29, 90, 13],
 [3, 9, 5, 177],
 [3, 1, 16, 13],
 [3, 70, 15, 71, 5],
 [68, 16, 13],
 [3, 9, 5],
 [19, 9, 5],
 [19, 1, 178],
 [3, 89, 15, 179, 18],
 [19, 89, 15, 180, 18],
 [3, 9, 5],
 [48, 6, 2],
 [3, 1, 2, 48],
 [181, 182],
 [26, 9, 5, 116, 92],
 [3, 9, 16, 93],
 [93, 6, 183],
 [26, 1, 4, 2, 92],
 [2, 48],
 [3, 21, 2, 48],
 [1, 2, 92, 94, 184],
 [49, 117, 21, 2, 48],
 [3, 1, 4, 2, 93],
 [26, 70, 15, 185, 18, 2],
 [26, 70, 15, 186, 2],
 

In [14]:
#ajouter des 0 aau debut de chaque ligne(liste) jusqu'a le longeur max de de grand liste
x_train = pad_sequences(train)
x_train

array([[  0,   0,   0, ...,   0,   0, 160],
       [  0,   0,   0, ...,  12,   9,   5],
       [  0,   0,   0, ...,   1, 161,  24],
       ...,
       [  0,   0,   0, ...,   0,   0,  17],
       [  0,   0,   0, ...,   0,  17,  13],
       [  0,   0,   0, ...,   1,   4,  17]])

# Encoding the output

In [15]:
data["tags"]

0      greeting
1      greeting
2      greeting
3      greeting
4      greeting
         ...   
400     ragging
401     ragging
402         hod
403         hod
404         hod
Name: tags, Length: 405, dtype: object

In [16]:
# convert categorical labels in a pandas DataFrame column into numerical values
le = LabelEncoder()
y_train = le.fit_transform(data["tags"])
y_train

array([14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 21, 21, 21,
       21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7, 12, 12, 12, 12, 12, 12, 12, 34, 34, 34, 34,
       34, 34, 34, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       18, 18, 18,  1,  1

In [17]:
x_train.shape

(405, 9)

In [18]:
input_shape = x_train.shape[1]
print(input_shape)

9


In [19]:
unique_words = len(tokenizer.word_index)
output_length = le.classes_.shape[0]
print("unique words: ", unique_words)
print("output length: ", output_length)

unique words:  277
output length:  38


In [22]:
model = tf.keras.Sequential()
model.add(Input(shape=(input_shape,)))
model.add(Embedding(unique_words+1, 10, input_length= (input_shape,)))

model.add(LSTM(10, return_sequences=True))
model.add(Flatten())
model.add(Dense(units= 10, activation='relu'))
model.add(Dense(units= 5, activation='relu'))
model.add(Dense(units= output_length, activation='softmax'))

In [24]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer ='adam', metrics=['accuracy'])

In [25]:
jarvis = model.fit(x_train, y_train, epochs=300)

Epoch 1/300
13/13 [==============================] - 2s 4ms/step - loss: 3.6362 - accuracy: 0.0420
Epoch 2/300
13/13 [==============================] - 0s 4ms/step - loss: 3.6305 - accuracy: 0.0519
Epoch 3/300
13/13 [==============================] - 0s 4ms/step - loss: 3.6227 - accuracy: 0.0494
Epoch 4/300
13/13 [==============================] - 0s 4ms/step - loss: 3.6104 - accuracy: 0.0494
Epoch 5/300
13/13 [==============================] - 0s 4ms/step - loss: 3.5924 - accuracy: 0.0420
Epoch 6/300
13/13 [==============================] - 0s 3ms/step - loss: 3.5720 - accuracy: 0.0642
Epoch 7/300
13/13 [==============================] - 0s 4ms/step - loss: 3.5431 - accuracy: 0.0642
Epoch 8/300
13/13 [==============================] - 0s 4ms/step - loss: 3.5055 - accuracy: 0.0642
Epoch 9/300
13/13 [==============================] - 0s 4ms/step - loss: 3.4610 - accuracy: 0.0642
Epoch 10/300
13/13 [==============================] - 0s 4ms/step - loss: 3.4041 - accuracy: 0.0642
Epoch 11/

13/13 [==============================] - 0s 4ms/step - loss: 1.6089 - accuracy: 0.4691
Epoch 84/300
13/13 [==============================] - 0s 4ms/step - loss: 1.5853 - accuracy: 0.4765
Epoch 85/300
13/13 [==============================] - 0s 4ms/step - loss: 1.5660 - accuracy: 0.4889
Epoch 86/300
13/13 [==============================] - 0s 3ms/step - loss: 1.5524 - accuracy: 0.4593
Epoch 87/300
13/13 [==============================] - 0s 3ms/step - loss: 1.5399 - accuracy: 0.4840
Epoch 88/300
13/13 [==============================] - 0s 4ms/step - loss: 1.5187 - accuracy: 0.4840
Epoch 89/300
13/13 [==============================] - 0s 4ms/step - loss: 1.5143 - accuracy: 0.4716
Epoch 90/300
13/13 [==============================] - 0s 4ms/step - loss: 1.4876 - accuracy: 0.5012
Epoch 91/300
13/13 [==============================] - 0s 3ms/step - loss: 1.4812 - accuracy: 0.5210
Epoch 92/300
13/13 [==============================] - 0s 4ms/step - loss: 1.4604 - accuracy: 0.5185
Epoch 93/300


13/13 [==============================] - 0s 4ms/step - loss: 0.2861 - accuracy: 0.9358
Epoch 246/300
13/13 [==============================] - 0s 4ms/step - loss: 0.3128 - accuracy: 0.9210
Epoch 247/300
13/13 [==============================] - 0s 4ms/step - loss: 0.3162 - accuracy: 0.9259
Epoch 248/300
13/13 [==============================] - 0s 4ms/step - loss: 0.2904 - accuracy: 0.9284
Epoch 249/300
13/13 [==============================] - 0s 4ms/step - loss: 0.2775 - accuracy: 0.9185
Epoch 250/300
13/13 [==============================] - 0s 4ms/step - loss: 0.2822 - accuracy: 0.9235
Epoch 251/300
13/13 [==============================] - 0s 3ms/step - loss: 0.2687 - accuracy: 0.9333
Epoch 252/300
13/13 [==============================] - 0s 4ms/step - loss: 0.2610 - accuracy: 0.9284
Epoch 253/300
13/13 [==============================] - 0s 4ms/step - loss: 0.2594 - accuracy: 0.9506
Epoch 254/300
13/13 [==============================] - 0s 4ms/step - loss: 0.2622 - accuracy: 0.9407
Epoc

In [ ]:
# num_words = 2000: le tokenizer ne conservera que les 2000 mots les plus fréquents dans vos données,
#en ignorant les mots moins fréquents.
tokenizer = Tokenizer(num_words = 2000)
# le tokenizer apprendra le vocabulaire à partir de ces données et attribuera un indice unique 
#à chaque mot dans le vocabulaire
tokenizer.fit_on_texts(data["patterns"])
#Chaque mot du texte est remplacé par son indice correspondant dans le vocabulaire 
#que le tokenizer a appris
train = tokenizer.texts_to_sequences(data["patterns"])
#ajouter des 0 aau debut de chaque ligne(liste) jusqu'a le longeur max de de grand liste
train

In [31]:

while True:
    textList = []
    user_input = input("You: ")
    prediction_input = []

    for letter in user_input:
        if letter not in string.punctuation:
            prediction_input.append(letter.lower())

    prediction_input = ''.join(prediction_input)
    textList.append(prediction_input)
    
    prediction_input = tokenizer.texts_to_sequences(textList)
    print(1,prediction_input)
    prediction_input = np.array(prediction_input).reshape(-1)
    print(2,prediction_input)
    prediction_input = pad_sequences([prediction_input], input_shape)
    print(3,prediction_input)
    output = model.predict(prediction_input)
    #print("output",output)
    output = output.argmax()
    print("output",output)
    response_tag = le.inverse_transform([output])[0]
    print("response_tag ",response_tag)
    print("Jarvis: ", random.choice(responses[response_tag]))
    if response_tag == 'goodbye':
        break

You: hello food
1 [[162, 64]]
2 [162  64]
3 [[  0   0   0   0   0   0   0 162  64]]
1/1 [==============================] - 0s 27ms/step
output 14
response_tag  greeting
Jarvis:  Hello!
You: hello , i was food 
1 [[162, 15, 64]]
2 [162  15  64]
3 [[  0   0   0   0   0   0 162  15  64]]
1/1 [==============================] - 0s 29ms/step
output 23
response_tag  name
Jarvis:  I am your helper
You: hello more info 
1 [[162, 188, 118]]
2 [162 188 118]
3 [[  0   0   0   0   0   0 162 188 118]]
1/1 [==============================] - 0s 20ms/step
output 13
response_tag  goodbye
Jarvis:  Come back soon


In [26]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


In [27]:
results = model.evaluate(x_val, y_val)
print("Loss:", results[0])
print("Accuracy:", results[1])

3/3 [==============================] - 0s 3ms/step - loss: 0.2331 - accuracy: 0.9630
Loss: 0.23309065401554108
Accuracy: 0.9629629850387573
